In [28]:
import json
import datetime
import re
import time
import numpy as np

from os import path
from scipy import stats
from yfpy.query import YahooFantasySportsQuery

from nba_api.stats.endpoints import PlayerGameLog
from nba_api.stats.static import players, teams

class Normal_Dist:
    def __init__(self, samples=None, num_samples=None, mean=None, variance=None, sd=None):
        if samples:
            self.num_samples = len(samples)
            self.mean = stats.tmean(samples)
            self.variance = stats.tvar(samples)
            self.sd = np.sqrt(self.variance)

        else:
            self.num_samples = num_samples
            self.mean = mean
            self.variance = variance
            self.sd = sd

    @classmethod
    def weighted_sum(cls, dist1, dist2, weight1, weight2):

        if dist1.num_samples == None:
            return dist2
        elif dist2.num_samples == None:
            return dist1

        num_samples = dist1.num_samples + dist2.num_samples
        mean = dist1.mean * weight1 + dist2.mean * weight2
        variance = dist1.variance * weight1 + dist2.variance * weight2 + weight1 * weight2 * (dist1.mean - dist2.mean)**2
        sd = np.sqrt(variance)

        return Normal_Dist(num_samples=num_samples, mean=mean, variance=variance, sd=sd)

class Player:
    def __init__(self, full_name, id, position):
        self.full_name = full_name
        self.id = id
        self.position = position
        self.categories = {}

    def calculate_dist(query, category):
        pass

    def get_category_dists(self, categories, w1=0.6, w2=0.4, min_mins=7, cume_stats_vs_teams=None, cume_stats_by_location=None):
        start = time.time()
        cume_stats = {c:[] for c in categories}
        cume_stats_prev_season = {c:[] for c in categories}

        for game in PlayerGameLog(player_id=self.id, season='2023').get_normalized_dict()["PlayerGameLog"]:
            if game['MIN'] < min_mins:
                continue
            _ ,location, opponent = game['MATCHUP'].split(' ')
            for cat in categories:
                cume_stats[cat].append(game[cat])
                if cume_stats_vs_teams:
                    cume_stats_vs_teams[opponent][cat].append(game[cat])
                if cume_stats_by_location:
                    cume_stats_by_location[location][cat].append(game[cat])
        
        for game in PlayerGameLog(player_id=self.id, season='2022').get_normalized_dict()["PlayerGameLog"]:
            if game['MIN'] < min_mins:
                continue
            _ ,location, opponent = game['MATCHUP'].split(' ')
            for cat in categories:
                cume_stats_prev_season[cat].append(game[cat])
                if cume_stats_vs_teams:
                    cume_stats_vs_teams[opponent][cat].append(game[cat])
                if cume_stats_by_location:
                    cume_stats_by_location[location][cat].append(game[cat])

        for cat in categories:
            current_season_dist = Normal_Dist(samples=cume_stats[cat])
            prev_season_dist = Normal_Dist(samples=cume_stats_prev_season[cat])
            self.categories[cat] = Normal_Dist.weighted_sum(prev_season_dist, current_season_dist, w1, w2)

        end = time.time()
        duration = end - start
        if duration < 0.6:
            time.sleep(0.6 - duration)

class Team:
    def __init__(self, owner_id):
        self.owner_id = owner_id
        self.players = []
        self.category_rankings = {}

class League:
    def __init__(self, auth_dir, league_id, game_id, categories):

        with open(path.join(auth_dir, r"private.json")) as f:
            private = json.load(f)
        self.yahoo_query = YahooFantasySportsQuery(
                                auth_dir,
                                league_id,
                                game_code= 'nba',
                                game_id=game_id,
                                offline=False,
                                all_output_as_json_str=False,
                                consumer_key=private["consumer_key"],
                                consumer_secret=private["consumer_secret"],
                                browser_callback=True)
        self.categories = categories
        
        self.cume_stats_vs_teams = {t['abbreviation']:{c:[] for c in categories} for t in teams.get_teams()}
        self.cume_stats_by_location = {l:{c:[] for c in categories} for l in ['vs.', '@']}
        self.players = [Player(p.full_name, find_nba_player_id(p.full_name), p.primary_position) for p in self.yahoo_query.get_league_players()]
        self.players = [p for p in self.players if p.id != None]
        for player in self.players:
            player.get_category_dists(categories, 0.5, 0.5, 7, self.cume_stats_vs_teams, self.cume_stats_by_location)

def find_nba_player_id(full_name):
    nba_players = players.get_players()

    search = [player for player in nba_players if player["full_name"] == full_name]
    if len(search) > 0:
        return search[0]['id']

    search = [player for player in nba_players if re.sub('\.|Jr|Sr|II|III|IV| ', '', player["full_name"]) == re.sub('\.|Jr|Sr|II|III|IV| ', '', full_name)]
    if len(search) > 0:
        return search[0]['id']

    else:
        return None

In [29]:
import cProfile

In [30]:
CATEGORIES = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'FG3M', 'FGM', 'FGA', 'FTM', 'FTA']
auth_dir = r"auth/"
league_id = r"101582"
game_id = 428
cProfile.run('league = League(auth_dir, league_id, game_id, CATEGORIES)', 'profilerstats')
# league = League(auth_dir, league_id, game_id, CATEGORIES)

2023-11-04 04:40:44.611 - ERROR - query.py - yfpy.query:289 - No data found when attempting extraction from fields: ['league', 'players']
c:\Users\chena\anaconda3\envs\fantasy\lib\site-packages\scipy\stats\_stats_py.py:667: RuntimeWarning: Degrees of freedom <= 0 for slice
  return a.var(ddof=ddof, axis=axis)
c:\Users\chena\anaconda3\envs\fantasy\lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [31]:
for p in league.players:
    print(p.full_name)
    for cat in CATEGORIES:
        print(cat, p.categories[cat].mean, p.categories[cat].sd)

LeBron James
PTS 26.754545454545458 7.747812472061676
REB 8.754545454545454 3.72053344911795
AST 6.709090909090909 2.210386063644088
STL 1.1545454545454545 1.076913591417272
BLK 0.7909090909090909 0.8928910133940291
TOV 3.8181818181818183 2.50312140612448
FG3M 2.2 1.7175564037317665
FGM 10.336363636363636 2.901899254582515
FGA 19.581818181818182 4.002813901582354
FTM 3.881818181818182 2.7208622753225384
FTA 5.872727272727273 3.3057295985541937
Andre Iguodala
PTS 2.125 2.295181287579947
REB 2.125 1.6420805617960927
AST 2.375 1.0606601717798212
STL 0.5 0.7559289460184544
BLK 0.375 0.5175491695067657
TOV 1.125 1.1259916264596033
FG3M 0.125 0.3535533905932738
FGM 0.875 0.8345229603962802
FGA 1.875 1.1259916264596033
FTM 0.25 0.7071067811865476
FTA 0.375 1.0606601717798212
Chris Paul
PTS 10.940677966101696 6.7844202684520845
REB 4.21045197740113 1.8677745773846282
AST 8.940677966101696 3.469295997793257
STL 1.6045197740112995 1.0944954508743385
BLK 0.1864406779661017 0.5070943667870021
TOV 

In [32]:
import pstats

In [33]:
pstats.Stats('profilerstats').strip_dirs().sort_stats('cumulative').print_stats()

Fri Nov  3 22:48:56 2023    profilerstats

         58873764 function calls (58745742 primitive calls) in 422.437 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  422.457  422.457 {built-in method builtins.exec}
        1    0.000    0.000  422.457  422.457 <string>:1(<module>)
        1    0.021    0.021  422.457  422.457 3807707865.py:99(__init__)
      614    0.215    0.000  397.919    0.648 3807707865.py:55(get_category_dists)
     1280    0.022    0.000  257.707    0.201 sessions.py:502(request)
     1280    0.030    0.000  255.717    0.200 sessions.py:673(send)
     1280    0.026    0.000  255.078    0.199 adapters.py:434(send)
     1280    0.027    0.000  253.735    0.198 connectionpool.py:596(urlopen)
     1280    0.028    0.000  253.419    0.198 connectionpool.py:381(_make_request)
     1228    0.006    0.000  252.323    0.205 playergamelog.py:16(__init__)
     1228    0.044    0.000  